In [45]:
from bs4 import BeautifulSoup
import requests as re
import pandas as pd

In [111]:
%%time
df = pd.DataFrame()
#url = input("http://www.e-stat.go.jp/SG1/estat/OtherListE.do?bid=000001006005&cycode=1")
url = "http://www.e-stat.go.jp/SG1/estat/OtherListE.do?bid=000001006005&cycode=1"
path=url[:url.rfind('/')]
r = re.get(url)
data = r.text
year='None'
soup = BeautifulSoup(data, "lxml")
for element in soup.find_all(['a','strong']):
    if(element.string != None and element.string.strip().strip(':').strip().isdigit()):
        year=str(element.string.strip().strip(":").strip())
#         print('year changed to ' + year)
    if(element.get('href') != None and "./" in element.get('href')):
        print(str(element.string).strip(".") + " " + year)
        print(path + element.get('href').strip('.'))

year changed to 2016
Jan 2016
http://www.e-stat.go.jp/SG1/estat/ListE.do?lid=000001144403
Feb 2016
http://www.e-stat.go.jp/SG1/estat/ListE.do?lid=000001148698
Mar 2016
http://www.e-stat.go.jp/SG1/estat/ListE.do?lid=000001151416
Apr 2016
http://www.e-stat.go.jp/SG1/estat/ListE.do?lid=000001153078
May 2016
http://www.e-stat.go.jp/SG1/estat/ListE.do?lid=000001153844
Jun 2016
http://www.e-stat.go.jp/SG1/estat/ListE.do?lid=000001154805
Jul 2016
http://www.e-stat.go.jp/SG1/estat/ListE.do?lid=000001157105
Aug 2016
http://www.e-stat.go.jp/SG1/estat/ListE.do?lid=000001158588
Sep 2016
http://www.e-stat.go.jp/SG1/estat/ListE.do?lid=000001160943
Oct 2016
http://www.e-stat.go.jp/SG1/estat/ListE.do?lid=000001162883
year changed to 2015
Jan 2015
http://www.e-stat.go.jp/SG1/estat/ListE.do?lid=000001129454
Feb 2015
http://www.e-stat.go.jp/SG1/estat/ListE.do?lid=000001132089
Mar 2015
http://www.e-stat.go.jp/SG1/estat/ListE.do?lid=000001133841
Apr 2015
http://www.e-stat.go.jp/SG1/estat/ListE.do?lid=00000

http://www.e-stat.go.jp/SG1/estat
